In [1]:
from google.colab import drive
drive.mount("/content/gdrive")

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [ ]:
!git clone https://github.com/tkipf/pygcn.git

Cloning into 'pygcn'...
remote: Enumerating objects: 78, done.
remote: Total 78 (delta 0), reused 0 (delta 0), pack-reused 78
Unpacking objects: 100% (78/78), done.


In [ ]:
%cd pygcn/

/content/gdrive/My Drive/Python/GitHub/pygcn/pygcn


In [7]:
from __future__ import division
from __future__ import print_function

import time
import argparse
import numpy as np

import torch
import torch.nn.functional as F
import torch.optim as optim

from pygcn.utils import load_data, accuracy
from pygcn.models import GCN

# Training settings

no_cuda = False
fastmode = False
seed = 42
epochs = 200
lr = .01
weight_decay = 5e-4
hidden = 16
dropout = 0.5

cuda = not no_cuda and torch.cuda.is_available()

np.random.seed(42)
torch.manual_seed(seed)
if cuda:
    torch.cuda.manual_seed(seed)

# Load data
adj, features, labels, idx_train, idx_val, idx_test = load_data()

# Model and optimizer
model = GCN(nfeat=features.shape[1],
            nhid=hidden,
            nclass=labels.max().item() + 1,
            dropout=dropout)
optimizer = optim.Adam(model.parameters(),
                       lr=lr, weight_decay=weight_decay)

if cuda:
    model.cuda()
    features = features.cuda()
    adj = adj.cuda()
    labels = labels.cuda()
    idx_train = idx_train.cuda()
    idx_val = idx_val.cuda()
    idx_test = idx_test.cuda()


def train(epoch):
    t = time.time()
    model.train()
    optimizer.zero_grad()
    output = model(features, adj)
    loss_train = F.nll_loss(output[idx_train], labels[idx_train])
    acc_train = accuracy(output[idx_train], labels[idx_train])
    loss_train.backward()
    optimizer.step()

    if not fastmode:
        # Evaluate validation set performance separately,
        # deactivates dropout during validation run.
        model.eval()
        output = model(features, adj)

    loss_val = F.nll_loss(output[idx_val], labels[idx_val])
    acc_val = accuracy(output[idx_val], labels[idx_val])
    print('Epoch: {:04d}'.format(epoch+1),
          'loss_train: {:.4f}'.format(loss_train.item()),
          'acc_train: {:.4f}'.format(acc_train.item()),
          'loss_val: {:.4f}'.format(loss_val.item()),
          'acc_val: {:.4f}'.format(acc_val.item()),
          'time: {:.4f}s'.format(time.time() - t))


def test():
    model.eval()
    output = model(features, adj)
    loss_test = F.nll_loss(output[idx_test], labels[idx_test])
    acc_test = accuracy(output[idx_test], labels[idx_test])
    print("Test set results:",
          "loss= {:.4f}".format(loss_test.item()),
          "accuracy= {:.4f}".format(acc_test.item()))


# Train model
t_total = time.time()
for epoch in range(epochs):
    train(epoch)
print("Optimization Finished!")
print("Total time elapsed: {:.4f}s".format(time.time() - t_total))

# Testing
test()


Loading cora dataset...
Epoch: 0001 loss_train: 1.9607 acc_train: 0.1143 loss_val: 1.9531 acc_val: 0.1133 time: 0.1731s
Epoch: 0002 loss_train: 1.9564 acc_train: 0.1214 loss_val: 1.9404 acc_val: 0.1133 time: 0.0059s
Epoch: 0003 loss_train: 1.9409 acc_train: 0.1357 loss_val: 1.9282 acc_val: 0.1133 time: 0.0052s
Epoch: 0004 loss_train: 1.9155 acc_train: 0.2214 loss_val: 1.9163 acc_val: 0.1300 time: 0.0051s
Epoch: 0005 loss_train: 1.9015 acc_train: 0.2500 loss_val: 1.9047 acc_val: 0.3567 time: 0.0051s
Epoch: 0006 loss_train: 1.9060 acc_train: 0.2357 loss_val: 1.8935 acc_val: 0.3500 time: 0.0053s
Epoch: 0007 loss_train: 1.8993 acc_train: 0.2857 loss_val: 1.8829 acc_val: 0.3500 time: 0.0055s
Epoch: 0008 loss_train: 1.8803 acc_train: 0.2929 loss_val: 1.8725 acc_val: 0.3500 time: 0.0054s
Epoch: 0009 loss_train: 1.8770 acc_train: 0.2929 loss_val: 1.8621 acc_val: 0.3500 time: 0.0050s
Epoch: 0010 loss_train: 1.8638 acc_train: 0.3214 loss_val: 1.8518 acc_val: 0.3500 time: 0.0049s
Epoch: 0011 loss